In [1]:
import geonamescache
gc = geonamescache.GeonamesCache()
internat_cities = [c for c in gc.get_cities().values() if c['population'] >= 100000 and c['countrycode'] != 'US']
import pycountry
countries = [c.name.title() for c in pycountry.countries]
planets = ['Mercury', 'Venus', 'Earth', 'Mars', 'Moon', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto'] #Pluto is not a planet
import requests

In [2]:
cit_file = "data/uscities.csv"
cities = []
with open(cit_file, "r") as f:
    k = 0
    for line in f:
        if k == 0:
            k=1
            continue
        
        row = [x.replace('\"', '') for x in line.split(',')]
        try:
            temp = {
                'name':row[0],
                'state':row[2],
                'longitude':float(row[8]),
                'latitude':float(row[9]),
                'population':int(round(float(row[10]))),
                'density':float(row[11])
            }
            cities.append(temp)
        except:
            temp = {
                'name':', '.join(row[0:2]),
                'state':row[4],
                'longitude':row[10],
                'latitude':row[11],
                'population':int(round(float(row[12]))),
                'density':float(row[13])
            }
            cities.append(temp)
city_names = {}
for c in cities:
    if c['name'] in city_names.keys():
        city_names[c['name']].append(c)
    else:
        city_names[c['name']] = [c]
for c in internat_cities:
    if c['name'] in city_names.keys():
        c['state'] = 'INTERNATIONAL'
        city_names[c['name']].append(c)

In [3]:
arc_name = "data/stream/2015/04/07"
save_name = "data/twitter_dump.txt"

In [4]:
abbr_to_states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state_to_abbr = {v.upper():k for k,v in abbr_to_states.items()}

In [5]:
wash_dc = [c for c in city_names['Washington'] if c['state']=='DC']
wash_dc = wash_dc[0]
nyc = city_names['New York'][0]

def match_country(st):
    if st.lower() in ['us', 'usa', 'america', 'united states', 'united states of america', 'amerika', 'murica', "'murica"]:
        return 'USA'
    else:
        return None

def match_state(st):
    stat = st.upper()
    if 'USA' in stat:
        stat = stat.replace('USA', '').strip()
    if 'AMERICA' in stat:
        stat = stat.replace('AMERICA', '').strip()
    if 'US' in stat:
        stat = stat.replace('US', '').strip()
    if stat in state_to_abbr.keys():
        return state_to_abbr[stat]
    elif stat not in state_to_abbr.values():
        return None
    return stat

def match_city(cit, st = None):
    city = cit.title()
    if city == 'New York City' or cit == 'Nyc':
        return nyc
    if st is None:
        if city in countries or city in planets:
            return None
        out = city_names.get(city, [])
        if len(out) > 1:
            temp = 0
            for c in out:
                if c['population']>temp:
                    temp = c['population']
                    oout = c
            if oout['state'] != 'INTERNATIONAL':
                return oout
            else:
                return None
        elif len(out)==1:
            return out[0]
        else:
            return None
    else:
        #print(cit, st)
        if st == 'DC':
            return wash_dc
        out = city_names.get(cit, None)
        if out:
            out = [c for c in out if c['state'] == st.upper()]
            if out:
                return out[0]
            else:
                return None
        else:
            return None

def match_string(st):
    st = st.strip()
    st = st.replace('.', '')
    if st.lower() == 'la':
        return match_string('Los Angeles, CA')
    if st.lower() == 'sf':
        return match_string('San Francisco, CA')
    if st.lower() == 'nyc':
        return nyc
    if 'nova iorque' in st.lower():
        return nyc
    if 'washington' in st.lower() and 'dc' in st.lower().replace('.',''):
        return wash_dc
    if st.count(',') == 0:
        try:
            out = match_country(st)
            if out:
                return out
        except:
            pass
        try:
            out = match_state(st)
            if out:
                return out
        except:
            pass
        try:
            out = match_city(st)
            if out:
                return out
        except:
            pass
        if st.count(' ') > 0:
            temp = st.split(' ')
            if len(temp) == 2:
                return match_string(', '.join(temp))
            else:
                return match_string(' '.join(temp[0:-1]) + ', ' + temp[-1])
        return None
    elif st.count(',') == 1:
        cit, st = st.split(',')
        cit = cit.strip().title()
        st = st.strip().upper()
        if match_country(st):
            if match_state(cit.upper()):
                return match_state(cit.upper())
            elif match_city(cit):
                return match_city(cit)
            else:
                return 'USA'
        elif match_state(st):
            st = match_state(st)
            if match_city(cit,st):
                return match_city(cit,st)
            else:
                """
                This is where I am going to do google geocoding to get the city
                """
                return None
                url = gmaps_url + cit + ',' + st + key_str
                response=requests.get(url)
                if response.status_code < 400:
                    results = response.json()['results']
                    if results:
                        results = results[0]
                        addr_components = [x['long_name'].title() for x in results['address_components']]
                        if cit in addr_components:
                            out = {
                                'state':st,
                                'name':cit,
                                'latitude':results['geometry']['location']['lat'],
                                'longitude':results['geometry']['location']['lng']
                            }
                            return out
                        else:
                            return st
                    else:
                        return st
                else:
                    return st

        else:
            return None
    elif st.count(',') == 2:
        cit, st, ct = st.split(',')
        if match_country(ct.strip()):
            return match_string(','.join([cit,st]))
        else:
            return None

In [6]:
def get_user_data(tweet):
    if (not tweet) or (not tweet.get('user', None)) or (not tweet['user']['location']):
        return None
    loc = match_string(tweet['user']['location'])
    if loc:
        if type(loc)==dict:
            out = [
                tweet['user']['id_str'],
                tweet['user']['friends_count'],
                tweet['user']['followers_count'],
                loc['state'],
                loc['name'],
                loc['latitude'],
                loc['longitude'],
                tweet['user']['created_at']
            ]
        else:
            out = [
                tweet['user']['id_str'],
                tweet['user']['friends_count'],
                tweet['user']['followers_count'],
                loc,
                None,
                None,
                None,
                tweet['user']['created_at']
            ]
        return out
    else:
        return None
    

In [7]:
from twitterArchiveParser import twitterArchiveParser

In [8]:
tp = twitterArchiveParser(archive_location=arc_name, save_location = save_name, num_threads = 7)

In [9]:
tp.parse_archive(get_user_data)

Thread finished

Thread finished

Thread finished

Thread finished

Thread finished

Thread finished

Thread finished

